<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/1_Models_%26_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
# import numpy as np
# import pandas as pd
# import seaborn as sn
# import matplotlib.pyplot as plt
# import textwrap

from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

In [ ]:
from tokenizers import Tokenizer
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
tokenizer("hello world")

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
#let's see behinde the seen of the above code:
tokens=tokenizer.tokenize("hello world")
tokens

['hello', 'world']

In [ ]:
ids=tokenizer.convert_tokens_to_ids(tokens)
ids

[7592, 2088]

In [ ]:
tokenizer.decode(ids)

'hello world'

In [ ]:
ids=tokenizer.encode("hello world")
ids

[101, 7592, 2088, 102]

In [ ]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'hello', 'world', '[SEP]']

In [ ]:
#bert similar to this code produce cls and sep
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [ ]:
model_inputs=tokenizer("hello world")
model_inputs

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
data=["I like cats.", "Do you like cats too?"]
tokenizer(data)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_inputs=tokenizer("hello world", return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [ ]:
#the model does not accept any kind of inputs and the defualt model for hugging face is pytorhch
#so we have to send list
#the defualt was to create a binary classifier
outputs=model(**model_inputs)
outputs
#the logits are not valid because the top layer of model have not been trained

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5564,  0.3946]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
#this time we have 3 labels for classification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs=model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1100, -0.1137,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
outputs.logits


tensor([[-0.1100, -0.1137,  0.0345]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs["logits"]

tensor([[-0.1100, -0.1137,  0.0345]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs[0]

tensor([[-0.1100, -0.1137,  0.0345]], grad_fn=<AddmmBackward0>)

In [ ]:
#if want to convert logits to numpy array it would be useful to compute accuracy
outputs.logits.detach().cpu().numpy()

array([[-0.10996927, -0.11373332,  0.03447594]], dtype=float32)

In [ ]:
data=["I like cats.", "Do you like cats too?"]

model_inputs=tokenizer(data,padding=True,truncation=True,return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
model_inputs['input_ids']

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])

In [ ]:
model_inputs["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
outputs=model(**model_inputs)
outputs
#the logits us 2*3 which means we have 2 documents and we have 3 labels

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2389, -0.0703,  0.1298],
        [ 0.1044, -0.0366,  0.0836]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)